In [62]:
import pandas as pd
import numpy as np
import warnings

In [58]:
#IMPORT DOS DADOS
df = (
    pd.read_csv("relatorios/tabela_ex3.csv", index_col=0)
    .query("year == 2016")
    .assign(
        participacao_industria = lambda _: (_.total_pais_industria / _.total_pais)
    )
)

In [59]:
df = (
    
    #CALCULO DO RANKING GLOBAL
    pd.concat([df
            .query(f'industry_name == "{industria}"')
            .sort_values(by = "RCA", ascending = False)
            .reset_index(drop = True).reset_index().rename({"index": "posicao_global"}, axis = 1)
        for industria in df.industry_name.unique()
        ]
    ).assign(
        posicao_global = lambda _: _.posicao_global + 1

    #ARRUMAR PARA O RELATÓRIO
    ).rename({
            "industry_name": "Indústria",
            "participacao_industria": "Participação da Indústria nas Exportações",
            "posicao_global": "Posição no Ranking Global RCA"
    }, axis = 1)
    .filter([
        "exporter_iso3",
        "Indústria", 
        "RCA", 
        "Participação da Indústria nas Exportações",
        "Posição no Ranking Global RCA", 
    ])
)

In [65]:
def relatorio_pais(pais = "BRA"):
    """Relatório em .tex para cada país"""

    return (df
        .query(f"exporter_iso3 == '{pais}'")
        .drop("exporter_iso3", axis = 1)
        .sort_values(by = "RCA", ascending = False)
        .head(6)
        .to_latex(
            index = False,
            caption = f"Relatório de exportações ({pais})",
            decimal = ",",
            formatters={
                'RCA': '{:.2f}'.format,
                "Participação da Indústria nas Exportações": '{:.2%}'.format
            },
            label = f"tab:ex3-{pais}",
            column_format = ''.join(['p{{{}cm}}'.format(width) for width in [6, 2, 4, 3]])
        )
    )

In [ ]:
#ESCRITA DE UM TEX POR PAIS
for pais in df.exporter_iso3.unique():
     with open(f'relatorios/tabelas/ex3-{pais}.tex', 'w') as tf:
          tf.write(relatorio_pais(pais = pais))

#TEX QUE INCLUI TODOS OS PAISES
with open(f'latex/arquivos/todas-tabelas.tex', 'w') as tf:
    tf.write("\n\n".join([f"\input{{../relatorios/tabelas/ex3-{pais}}}" for pais in df.sort_values(by = "exporter_iso3").exporter_iso3.unique()]))

warnings.filterwarnings("ignore", category=DeprecationWarning)